In [227]:
import pandas as pd
import numpy as np
import re
import time
import dill
from datetime import timedelta
from csv_pkl_sql import save_it, pkl_it

## Scrape appropriate date and location for weather data
First requires finding closest airport for each location.

In [228]:
lat_long_data = pd.read_pickle('../pkl/01_latitude_longitude_google.pkl')
lat_long_data.head(1)

,location,latitude,longitude
0,Argentina-Buenos_Aires,-34.603684,-58.381559


In [229]:
airport_info = pd.read_pickle('../pkl/02_airport_information_fallingrain.pkl')
airport_info.head(1)

,city,FAA,IATA,ICAO,kind,latitude,longitude,max_runway,name,country,state
56,BAHIA BLANCA,NaN,BHI,SAZB,Medium,-38.725,-62.169,8579.0,COMANDANTE ESPORA,Argentina,NaN


The approximation for closest airport is crude, given that it doesn't convert latitude and longitude to distance but rather uses them directly. Given the relatively short distances involved, I think this is fine for a first pass of this project.

In [171]:
airport_coords = airport_info[['latitude', 'longitude']].values[np.newaxis, :]
places_coords = np.rollaxis(lat_long_data[['latitude','longitude']].values[np.newaxis, :], 0, -1)

dist_coords = ((places_coords - airport_coords)**2).sum(axis=-1)
min_coords = dist_coords.argmin(axis=1)

print airport_coords.shape, places_coords.shape, dist_coords.shape, min_coords.shape

(1, 2062, 2) (1606, 1, 2) (1606, 2062) (1606,)


In [172]:
# Transfer the coordinates to the latitude/longitude data
merge_data = lat_long_data.copy()

print merge_data.shape

merge_data['airport_index'] = airport_info.index[min_coords]

# Now grap the airport and location info
df = airport_info.loc[merge_data.airport_index, ['country','name','FAA','IATA','ICAO']]
merge_data[['country','name','FAA','IATA','ICAO']] = df.set_index(merge_data.index)

print merge_data.shape

(1606, 3)
(1606, 9)


In [173]:
merge_data.head()

,location,latitude,longitude,airport_index,country,name,FAA,IATA,ICAO
0,Argentina-Buenos_Aires,-34.603684,-58.381559,80,Argentina,AEROPARQUE JORGE NEWBERY,NaN,AEP,SABE
1,Argentina-CABA,-34.603684,-58.381559,80,Argentina,AEROPARQUE JORGE NEWBERY,NaN,AEP,SABE
2,Argentina-Cordoba,-31.420083,-64.188776,149,Argentina,AMBROSIO L V TARAVELLA,NaN,COR,SACO
3,Argentina-Entre_Rios,-31.774665,-60.495646,398,Argentina,GENERAL URQUIZA,NaN,PRA,SAAP
4,Argentina-Santa_Fe,-31.610658,-60.697294,527,Argentina,SAUCE VIEJO,NaN,SFN,SAAV


In [174]:
# TODO WRITE THIS MATRIX OUT
pkl_it(merge_data, '04_merged_latitude_longitude_airport_checkpoint')

Now combine with infection date data.

In [230]:
infection_data = pd.read_pickle('../pkl/03_infection_data_initial_import.pkl')
infection_data = infection_data[['date','location']]
infection_data.head(1)

,date,location
0,2016-03-19,Argentina-Buenos_Aires


In [178]:
print infection_data.shape, merge_data.shape

merge_all = pd.merge(infection_data, 
                     merge_data[['location','country','FAA','IATA','ICAO']], 
                     on='location', 
                     how='left').drop_duplicates()

print merge_all.shape

merge_all.head()

(105374, 2) (1606, 9)
(34440, 6)


,date,location,country,FAA,IATA,ICAO
0,2016-03-19,Argentina-Buenos_Aires,Argentina,NaN,AEP,SABE
5,2016-03-19,Argentina-CABA,Argentina,NaN,AEP,SABE
10,2016-03-19,Argentina-Catamarca,Argentina,NaN,CTC,SANC
15,2016-03-19,Argentina-Chaco,Argentina,NaN,RES,SARE
20,2016-03-19,Argentina-Chubut,Argentina,NaN,REL,SAVT


Now scrape from weather underground. I want time shifted data, so need to get one and two weeks beforehand.

In [181]:
weather_scrape = (merge_all[['date','country','IATA','ICAO']]
                  .drop_duplicates()
                  .set_index(['country','IATA','ICAO'])
                  )

weather_scrape['date1'] = weather_scrape.date - timedelta(days=7)
weather_scrape['date2'] = weather_scrape.date - timedelta(days=14)

weather_scrape = (weather_scrape
                  .stack()
                  .reset_index(level=-1, drop=True)
                  .reset_index()
                  .rename(columns={0:'date'})
                  .dropna(subset=['IATA','ICAO'], how='all')
                 )

weather_scrape.shape

(15054, 4)

In [182]:
# def scrape_weekly_weather(df_row):
#     # Scrape the weekly data table
#     url_fmt = 'https://www.wunderground.com/history/airport/{}/{}/{}/{}/WeeklyHistory.html'
    
#     try:
#         url = url_fmt.format(df_row.ICAO, df_row.report_date.year, 
#                              df_row.report_date.month, df_row.report_date.day)
#     except:
#         url = url_fmt.format(df_row.IATA, df_row.report_date.year, 
#                              df_row.report_date.month, df_row.report_date.day)
    
#     try:
#         table = pd.read_html(url)[0].dropna(subset=['Max','Avg','Min','Sum'], how='all')
#         table.columns = ['Measurement','Max','Avg','Min','Sum']
#         table.set_index('Measurement', inplace=True)
#         table = table.stack()
#     except:
#         table = pd.Series({'NULL':np.NaN}, index=pd.Index([0]))
    
#     return table

def scrape_weekly_weather(date, df_row):
    # Scrape the weekly data table
    url_fmt = 'https://www.wunderground.com/history/airport/{}/{}/{}/{}/WeeklyHistory.html'
    
    try:
        url = url_fmt.format(df_row.ICAO, date.year, 
                             date.month, date.day)
    except:
        url = url_fmt.format(df_row.IATA, date.year, 
                             date.month, date.day)
    
    try:
        table = pd.read_html(url)[0].dropna(subset=['Max','Avg','Min','Sum'], how='all')
        table.columns = ['Measurement','Max','Avg','Min','Sum']
        table.set_index('Measurement', inplace=True)
        table = table.stack()
        time.sleep(1.0)
    except:
        table = pd.Series({'NULL':np.NaN}, index=pd.Index([0]))
    
    return table

In [ ]:
# Broke up into sections and did this on three different computers to speed it up
for ndate, date in enumerate(date_list):
    
    print ndate
    df_list = list()
    
    for num,(row,dat) in enumerate(airport_list.iterrows()):
        
        try:
            df = scrape_weekly_weather(date, dat)
        except:
            df = pd.Series({'NULL':np.NaN}, index=pd.Index([row]))

        df_list.append((date, dat.name, df))
        
    with open('../pkl/df_list{}.pkl'.format(ndate),'w') as fh:
        dill.dump(df_list, fh)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


In [125]:
def clean_weather_data(entry):
    index = pd.MultiIndex.from_tuples([(entry[0],
                                        entry[1])]*len(entry[2]),
                                      names=['date','index'])
    
    df = pd.DataFrame(entry[2].reset_index().values, 
                      index=index, 
                      columns=['measurement','type','value'])

    mask = (df.measurement.isin(['Max Temperature','Mean Temperature',
                                   'Min Temperature','Dew Point','Precipitation','Wind']))
    df = df.loc[mask]
    
    mask = ((((df.measurement=='Precipitation')&(df.type=='Sum'))|(df.type=='Avg')) & 
            ((df.measurement=='Precipitation')&(df.type=='Avg')).pipe(np.invert))
    df = df.loc[mask].drop(['type'], axis=1)
    
    df['value'] = (df.value
                   .str.replace('-', '')
                   .str.extract(r"""([0-9.-]+)""", expand=True)
                   .astype(float)
                   )
    
    return df

df_clean = list()


for i in range(134):
    with open('../pkl/df_list{}.pkl'.format(i), 'r') as fh:
        df_list = dill.load(fh)
    
    for df in enumerate(df_list):
        if not df[1][2].isnull().all():
            df_clean.append(clean_weather_data(df[1]))

In [144]:
weather_combined = pd.concat(df_clean, axis=0)
weather_combined.head()

measurement  value
date       index                         
2015-11-14 0       Max Temperature  76.00
           0      Mean Temperature  70.00
           0       Min Temperature  63.00
           0             Dew Point  58.00
           0         Precipitation   3.31

In [145]:
weather_combined = pd.merge(weather_combined.reset_index(level=-1), 
                            airport_list, 
                            left_on='index', 
                            right_index=True).drop(['index'], axis=1).reset_index()

## Shift historical weather data

In [146]:
def time_shift(df, feature, week=1):
    new_df = (pd.merge(df[['date', feature]].reset_index(),
                       df[['date'+str(week), feature]].reset_index(),
                       left_on=df.index.names + ['date'], 
                       right_on=df.index.names + ['date'+str(week)],
                       suffixes=('',str(week)), 
                       how='inner')
              .drop(['date'+str(week)] + df.index.names, axis=1)
              .reset_index(level=-1, drop=True))
        
    return new_df


def create_weather_feature(df, feature):
    df_new = (df.loc[df.measurement==feature]
             .set_index(['ICAO','IATA','date','measurement'])
             .unstack())
    
    df_new = df_new.reset_index(level=-1)
    df_new.columns = ['date', feature]

    df_new['date1'] = df_new.date + timedelta(days=7)
    df_new['date2'] = df_new.date + timedelta(days=14)

    df_new1 = (df_new
            .groupby(level=[0,1])
            .apply(lambda x: time_shift(x,feature, 1))
            .reset_index(level=-1,drop=True))
    
    df_new2 = (df_new
            .groupby(level=[0,1])
            .apply(lambda x: time_shift(x, feature, 2))
            .reset_index(level=-1,drop=True))
    
    df_new = pd.merge(df_new1.reset_index(),
                      df_new2.reset_index().drop([feature], axis=1),
                      on=df_new1.index.names + ['date']).set_index(df_new1.index.names)
    
    return df_new

In [148]:
# Shift the one and two week prior data

max_temp = create_weather_feature(weather_combined, 'Max Temperature').set_index('date',append=True)
mean_temp = create_weather_feature(weather_combined, 'Mean Temperature').set_index('date',append=True)
min_temp = create_weather_feature(weather_combined, 'Min Temperature').set_index('date',append=True)
dew_point = create_weather_feature(weather_combined, 'Dew Point').set_index('date',append=True)
precipitation = create_weather_feature(weather_combined, 'Precipitation').set_index('date',append=True)
wind = create_weather_feature(weather_combined, 'Wind').set_index('date',append=True)

In [149]:
[x.shape for x in max_temp, mean_temp, min_temp, dew_point, precipitation, wind]

[(19213, 3), (19213, 3), (19213, 3), (19213, 3), (19213, 3), (19213, 3)]

In [150]:
[x.isnull().sum().max() for x in max_temp, mean_temp, min_temp, dew_point, precipitation, wind]

[733, 811, 722, 758, 510, 787]

Impute missing data.

In [151]:
max_temp = max_temp.interpolate(method='linear', limit_direction='both')
mean_temp = mean_temp.interpolate(method='linear', limit_direction='both')
min_temp = min_temp.interpolate(method='linear', limit_direction='both')
dew_point = dew_point.interpolate(method='linear', limit_direction='both')
precipitation = precipitation.interpolate(method='linear', limit_direction='both')
wind = wind.interpolate(method='linear', limit_direction='both')

In [152]:
[x.shape for x in max_temp, mean_temp, min_temp, dew_point, precipitation, wind]

[(19213, 3), (19213, 3), (19213, 3), (19213, 3), (19213, 3), (19213, 3)]

In [153]:
[x.isnull().sum().max() for x in max_temp, mean_temp, min_temp, dew_point, precipitation, wind]

[0, 0, 0, 0, 0, 0]

Add location key.

In [231]:
airport = pd.read_pickle('../pkl/04_merged_latitude_longitude_airport_checkpoint.pkl')
airport.head(1)

,location,latitude,longitude,airport_index,country,name,FAA,IATA,ICAO
0,Argentina-Buenos_Aires,-34.603684,-58.381559,80,Argentina,AEROPARQUE JORGE NEWBERY,NaN,AEP,SABE


In [155]:
max_temp = pd.merge(max_temp.reset_index(),
         airport[['ICAO','IATA','location']],
         on=['ICAO','IATA'],
         how='left')#.drop_duplicates(subset=['location'])

mean_temp = pd.merge(mean_temp.reset_index(),
         airport[['ICAO','IATA','location']],
         on=['ICAO','IATA'],
         how='left')


min_temp = pd.merge(min_temp.reset_index(),
         airport[['ICAO','IATA','location']],
         on=['ICAO','IATA'],
         how='left')


dew_point = pd.merge(dew_point.reset_index(),
         airport[['ICAO','IATA','location']],
         on=['ICAO','IATA'],
         how='left')

precipitation = pd.merge(precipitation.reset_index(),
         airport[['ICAO','IATA','location']],
         on=['ICAO','IATA'],
         how='left')

wind = pd.merge(wind.reset_index(),
         airport[['ICAO','IATA','location']],
         on=['ICAO','IATA'],
         how='left')

In [156]:
[x.shape for x in max_temp, mean_temp, min_temp, dew_point, precipitation, wind]

[(108373, 7), (108373, 7), (108373, 7), (108373, 7), (108373, 7), (108373, 7)]

In [157]:
[x.isnull().sum().max() for x in max_temp, mean_temp, min_temp, dew_point, precipitation, wind]

[0, 0, 0, 0, 0, 0]

In [158]:
max_temp = max_temp.drop(['ICAO','IATA'], axis=1).drop_duplicates(subset=['location','date'])
max_temp.columns = [x.lower().replace(' ', '_').replace('erature','') for x in max_temp.columns]

mean_temp = mean_temp.drop(['ICAO','IATA'], axis=1).drop_duplicates(subset=['location','date'])
mean_temp.columns = [x.lower().replace(' ', '_').replace('erature','') for x in mean_temp.columns]

min_temp = min_temp.drop(['ICAO','IATA'], axis=1).drop_duplicates(subset=['location','date'])
min_temp.columns = [x.lower().replace(' ', '_').replace('erature','') for x in min_temp.columns]

dew_point = dew_point.drop(['ICAO','IATA'], axis=1).drop_duplicates(subset=['location','date'])
dew_point.columns = [x.lower().replace(' ', '_').replace('erature','') for x in dew_point.columns]

precipitation = precipitation.drop(['ICAO','IATA'], axis=1).drop_duplicates(subset=['location','date'])
precipitation.columns = [x.lower().replace(' ', '_').replace('erature','') for x in precipitation.columns]

wind = wind.drop(['ICAO','IATA'], axis=1).drop_duplicates(subset=['location','date'])
wind.columns = [x.lower().replace(' ', '_').replace('erature','') for x in wind.columns]

In [159]:
[x.shape for x in max_temp, mean_temp, min_temp, dew_point, precipitation, wind]

[(107965, 5), (107965, 5), (107965, 5), (107965, 5), (107965, 5), (107965, 5)]

In [160]:
[x.isnull().sum().max() for x in max_temp, mean_temp, min_temp, dew_point, precipitation, wind]

[0, 0, 0, 0, 0, 0]

In [161]:
print max_temp.shape

weather_final = pd.merge(max_temp, mean_temp, on=['date','location'], how='inner')
weather_final = pd.merge(weather_final, min_temp, on=['date','location'], how='inner')
weather_final = pd.merge(weather_final, dew_point, on=['date','location'], how='inner')
weather_final = pd.merge(weather_final, precipitation, on=['date','location'], how='inner')
weather_final = pd.merge(weather_final, wind, on=['date','location'], how='inner')

print weather_final.shape

(107965, 5)
(107965, 20)


In [162]:
weather_final.isnull().sum().max()

0

In [164]:
weather_final.head(1).T

,0
date,2015-11-28 00:00:00
max_temp,67
max_temp1,70
max_temp2,68
location,United_States-Florida-Columbia_County
mean_temp,53
mean_temp1,57
mean_temp2,56
min_temp,38
min_temp1,43


In [165]:
weather_final.dtypes

date              datetime64[ns]
max_temp                 float64
max_temp1                float64
max_temp2                float64
location                  object
mean_temp                float64
mean_temp1               float64
mean_temp2               float64
min_temp                 float64
min_temp1                float64
min_temp2                float64
dew_point                float64
dew_point1               float64
dew_point2               float64
precipitation            float64
precipitation1           float64
precipitation2           float64
wind                     float64
wind1                    float64
wind2                    float64
dtype: object

In [167]:
save_it(weather_final, '04_weekly_weather')